Dataset

In [6]:
import os
from PIL import Image
import json

def load_images_from_directory(root_path: str):
    """
    Load images from a directory with subfolders named after ImageNet labels.
    Return a list of (image, label, filename) triples.
    """
    dataset = []
    
    # Iterate over each subfolder
    for label in os.listdir(root_path):
        label_path = os.path.join(root_path, label)
        
        # Check if it's indeed a folder
        if os.path.isdir(label_path):
            
            # Iterate over each image in the subfolder
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                
                # Check if it's an image file
                if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = Image.open(image_path)
                    dataset.append((img, label, image_file))  # Add image filename here
    
    return dataset


current_dir = "/home/workstation/code/XAImethods/hf_cam_dev"

dataset_path = f"{current_dir}/ImageNet-Mini/images"
dataset = load_images_from_directory(dataset_path)


with open(f"{current_dir}/ImageNet-Mini/imagenet_class_index.json", "r") as f:
    imagenet_class_index = json.load(f)


label_to_index_description = {v[0]: (k, v[1]) for k, v in imagenet_class_index.items()}


[codecarbon INFO @ 13:27:17] Energy consumed for RAM : 0.000730 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:27:17] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:27:17] Energy consumed for all CPUs : 0.005937 kWh. All CPUs Power : 47.5 W
[codecarbon INFO @ 13:27:17] 0.006667 kWh of electricity used since the begining.
[codecarbon INFO @ 13:27:32] Energy consumed for RAM : 0.000754 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:27:32] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:27:32] Energy consumed for all CPUs : 0.006135 kWh. All CPUs Power : 47.5 W
[codecarbon INFO @ 13:27:32] 0.006889 kWh of electricity used since the begining.
[codecarbon INFO @ 13:27:47] Energy consumed for RAM : 0.000778 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:27:47] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:27:47] Energy consumed f

functions

In [2]:
import warnings
warnings.filterwarnings('ignore')
from codecarbon import EmissionsTracker
from torchvision import transforms
from datasets import load_dataset
from pytorch_grad_cam import run_dff_on_image
from pytorch_grad_cam import (
    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA!")
else:
    device = torch.device("cpu")
    print("Using CPU!")
# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]
# img_tensor = transforms.ToTensor()(image)

""" Model wrapper to return a tensor"""
class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

""" Translate the category name to the category index.
    Some models aren't trained on Imagenet but on even larger datasets,
    so we can't just assume that 761 will always be remote-control.

"""
def category_name_to_index(model, category_name):
    name_to_index = dict((v, k) for k, v in model.config.id2label.items())
    return name_to_index[category_name]
    
""" Helper function to run GradCAM on an image and create a visualization.
    (note to myself: this is probably useful enough to move into the package)
    If several targets are passed in targets_for_gradcam,
    e.g different categories,
    a visualization for each of them will be created.
    
"""
# def run_grad_cam_on_image(model: torch.nn.Module,
#                           target_layer: torch.nn.Module,
#                           targets_for_gradcam: List[Callable],
#                           reshape_transform: Optional[Callable],
#                           input_tensor: torch.nn.Module=img_tensor,
#                           input_image: Image=image,
#                           method: Callable=GradCAM):
#     with method(model=HuggingfaceToTensorModelWrapper(model),
#                  target_layers=[target_layer],
#                  reshape_transform=reshape_transform) as cam:

#         # Replicate the tensor for each of the categories we want to create Grad-CAM for:
#         repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

#         batch_results = cam(input_tensor=repeated_tensor,
#                             targets=targets_for_gradcam)
#         results = []
#         for grayscale_cam in batch_results:
#             visualization = show_cam_on_image(np.float32(input_image)/255,
#                                               grayscale_cam,
#                                               use_rgb=True)
#             # Make it weight less in the notebook:
#             visualization = cv2.resize(visualization,
#                                        (visualization.shape[1]//2, visualization.shape[0]//2))
#             results.append(visualization)
#         return np.hstack(results)
    

# Define the CAM algorithm you want to use
# Options: GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, 
# EigenCAM, EigenGradCAM, LayerCAM, FullGrad, GradCAMElementWise
CAM_ALGORITHM = GradCAM
cam_algorithm_name = CAM_ALGORITHM.__name__
    
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]



Using CUDA!


apple/mobilevit-small

In [3]:
# import os
# from tqdm import tqdm
# from collections import defaultdict
# import gc

# # Import the necessary modules
# from transformers import MobileViTForImageClassification

# tracker = EmissionsTracker()
# tracker.start()

# # Load the model and send it to the appropriate device
# model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-small").to(device)
# target_layer = model.mobilevit.conv_1x1_exp

# # Define the transformation to convert images to tensor
# transform = transforms.ToTensor()

# # Define the directory to save results
# save_dir = "/home/workstation/code/XAImethods/pytorch-grad-cam/apple/mobilevit-small"
# # Ensure the directory exists
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# # Convert any grayscale image to RGB
# def ensure_rgb(img):
#     if img.mode != 'RGB':
#         return img.convert('RGB')
#     return img

# # Iterate over all images in the dataset
# for idx, (img, label, filename) in tqdm(enumerate(dataset), total=len(dataset)):

#     torch.cuda.empty_cache()
#     img = ensure_rgb(img)  # Ensure the image is in RGB format
#     img_tensor = transform(img).to(device)
#     # Dynamically generate targets_for_gradcam based on model's prediction
#     #dynamic_targets_for_gradcam = get_top_k_targets(model, img_tensor)

#     # 使用标签获取索引和描述
#     index_description = label_to_index_description.get(label)
#     if index_description is None:
#         print(f"Warning: Label '{label}' not found in the JSON file!")
#         continue
    
#     index_str, description = index_description
#     index = int(index_str)  # 将字符串索引转换为整数

#     dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

#     gradcam_result, grayscale_cams = run_grad_cam_on_image(
#         model=model,
#         target_layer=target_layer,
#         targets_for_gradcam=dynamic_targets_for_gradcam,
#         input_tensor=img_tensor,
#         input_image=img,
#         reshape_transform=None
#     )

#     # Extract top predictions
#     logits = model(img_tensor.unsqueeze(0)).logits
#     top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
#     predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
    
#     # Define individual directory for this image using the original filename
#     img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])  # Remove file extension from filename
#     if not os.path.exists(img_dir):
#         os.makedirs(img_dir)
    
#     # Define file names
#     img_name = os.path.join(img_dir, "original.jpg")
#     gradcam_name = os.path.join(img_dir, "gradcam.jpg")
#     grayscale_name = os.path.join(img_dir, "grayscale.jpg")
#     grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
#     scores_name = os.path.join(img_dir, "scores.npy")
#     info_name = os.path.join(img_dir, "info.txt")

#     # Save the images and results
#     img.save(img_name)
#     Image.fromarray(gradcam_result).save(gradcam_name)
#     Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
#     np.save(grayscale_npy_name, grayscale_cams[0])

#     # Save the scores
#     scores = [data["score"] for _, data in predictions.items()]
#     np.save(scores_name, scores)

#     # Save the other info
#     with open(info_name, 'w') as f:
#         for index, data in predictions.items():
#             label = data["label"]
#             score = data["score"]
#             f.write(f"Class {index} ({label}): {score:.2f}\n")

#     del img, label, img_name, img_tensor, gradcam_result, grayscale_cams, logits, top_indices, predictions, scores, index_description, index_str, description, index, dynamic_targets_for_gradcam
#     torch.cuda.empty_cache()
#     gc.collect()

# tracker.stop()
# emissions_data = tracker.get_emissions()

# output_dir = "/home/workstation/code/XAImethods/pytorch-grad-cam/apple"
# tracker = EmissionsTracker(output_dir=output_dir)

# print(emissions_data)

try batch size


In [4]:
# import os
# from tqdm import tqdm
# from collections import defaultdict
# import gc
# from transformers import MobileViTForImageClassification
# import os
# from PIL import Image

# MAX_FILE_SIZE = 5 * 1024 * 1024  # e.g., 5 MB

# def is_valid_image_file(filepath):
#     """Check if the file is a valid image file."""
#     try:
#         with Image.open(filepath) as img:
#             img.verify()  # verify that it is a valid image
#         return True
#     except:
#         return False
    
# tracker = EmissionsTracker()
# tracker.start()

# BATCH_SIZE = 100

# # Split the dataset into batches
# num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

# for batch_num in range(num_batches):
#     start_idx = batch_num * BATCH_SIZE
#     end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

#     for idx in range(start_idx, end_idx):
#         data = dataset[idx]

#         try:


#             # Load the model and send it to the appropriate device
#             model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-small").to(device)
#             target_layer = model.mobilevit.conv_1x1_exp

#             # Define the transformation to convert images to tensor
#             transform = transforms.ToTensor()

#             # Define the directory to save results
#             save_dir = "/home/workstation/code/XAImethods/pytorch-grad-cam/apple/mobilevit-small"
#             # Ensure the directory exists
#             if not os.path.exists(save_dir):
#                 os.makedirs(save_dir)
#             # Convert any grayscale image to RGB
#             def ensure_rgb(img):
#                 if img.mode != 'RGB':
#                     return img.convert('RGB')
#                 return img

#             # Iterate over all images in the dataset
#             for idx, (img, label, filename) in tqdm(enumerate(dataset), total=len(dataset)):

#                 torch.cuda.empty_cache()
#                 img = ensure_rgb(img)  # Ensure the image is in RGB format
#                 img_tensor = transform(img).to(device)
#                 # Dynamically generate targets_for_gradcam based on model's prediction
#                 #dynamic_targets_for_gradcam = get_top_k_targets(model, img_tensor)

#                 # 使用标签获取索引和描述
#                 index_description = label_to_index_description.get(label)
#                 if index_description is None:
#                     print(f"Warning: Label '{label}' not found in the JSON file!")
#                     continue
                
#                 index_str, description = index_description
#                 index = int(index_str)  # 将字符串索引转换为整数

#                 dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

#                 gradcam_result, grayscale_cams = run_grad_cam_on_image(
#                     model=model,
#                     target_layer=target_layer,
#                     targets_for_gradcam=dynamic_targets_for_gradcam,
#                     input_tensor=img_tensor,
#                     input_image=img,
#                     reshape_transform=None
#                 )

#                 # Extract top predictions
#                 logits = model(img_tensor.unsqueeze(0)).logits
#                 top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
#                 predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
                
#                 # Define individual directory for this image using the original filename
#                 img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])  # Remove file extension from filename
#                 if not os.path.exists(img_dir):
#                     os.makedirs(img_dir)
                
#                 # Define file names
#                 img_name = os.path.join(img_dir, "original.jpg")
#                 gradcam_name = os.path.join(img_dir, "gradcam.jpg")
#                 grayscale_name = os.path.join(img_dir, "grayscale.jpg")
#                 grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
#                 scores_name = os.path.join(img_dir, "scores.npy")
#                 info_name = os.path.join(img_dir, "info.txt")

#                 # Save the images and results
#                 img.save(img_name)
#                 Image.fromarray(gradcam_result).save(gradcam_name)
#                 Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
#                 np.save(grayscale_npy_name, grayscale_cams[0])

#                 # Save the scores
#                 scores = [data["score"] for _, data in predictions.items()]
#                 np.save(scores_name, scores)

#                 # Save the other info
#                 with open(info_name, 'w') as f:
#                     for index, data in predictions.items():
#                         label = data["label"]
#                         score = data["score"]
#                         f.write(f"Class {index} ({label}): {score:.2f}\n")
#         except RuntimeError as e:
#             if "CUDA out of memory" in str(e):
#                 print(f"CUDA OutOfMemoryError encountered for file: {filename}")
#                 continue
#             else:
#                 raise e

#         #del img, label, img_name, img_tensor, gradcam_result, grayscale_cams, logits, top_indices, predictions, scores, index_description, index_str, description, index, dynamic_targets_for_gradcam
#         torch.cuda.empty_cache()
#     # Clean up after processing the batch
#     del model
#     torch.cuda.empty_cache()
#     gc.collect()

#     # Reload the model for the next batch
#     model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-small").to(device)


# torch.cuda.empty_cache()


# tracker.stop()
# emissions_data = tracker.get_emissions()

# output_dir = "/home/workstation/code/XAImethods/pytorch-grad-cam/apple"
# tracker = EmissionsTracker(output_dir=output_dir)

# print(emissions_data)

In [5]:
import os
from tqdm import tqdm
from collections import defaultdict
import gc
from transformers import MobileViTForImageClassification
from PIL import Image
import datetime

from codecarbon import EmissionsTracker

model_name = "apple/mobilevit-small"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []

tracker = EmissionsTracker()
tracker.start()


def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = MobileViTForImageClassification.from_pretrained("apple/mobilevit-small").to(device)
    target_layer = model.mobilevit.conv_1x1_exp
    transform = transforms.ToTensor()

    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            img_tensor = transform(img).to(device)

            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=img_tensor,
                input_image=img,
                reshape_transform=None
            )

            logits = model(img_tensor.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()

# After processing
tracker.stop()
end_time = datetime.datetime.now()


# Get emissions data
emissions_data = tracker.get_emissions()

# Save emissions report
report_filename = os.path.join(save_dir, "emissions_report.txt")
with open(report_filename, 'w') as f:
    json.dump(emissions_data, f, indent=4)

# Print summary
print(f"Processing started at: {start_time}")
print(f"Processing ended at: {end_time}")
print(f"Duration: {end_time - start_time}")
print(f"Emissions report saved to: {report_filename}")



[codecarbon INFO @ 13:19:42] [setup] RAM Tracking...
[codecarbon INFO @ 13:19:42] [setup] GPU Tracking...
[codecarbon INFO @ 13:19:42] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:19:42] [setup] CPU Tracking...
[codecarbon WARNING @ 13:19:42] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:19:44] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz
[codecarbon INFO @ 13:19:44] >>> Tracker's metadata:
[codecarbon INFO @ 13:19:44]   Platform system: Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 13:19:44]   Python version: 3.10.9
[codecarbon INFO @ 13:19:44]   Available RAM : 15.576 GB
[codecarbon INFO @ 13:19:44]   CPU count: 16
[codecarbon INFO @ 13:19:44]   CPU model: Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz
[codecarbon INFO @ 13:19:44]   GPU count: 1
[codecarbon INFO @ 13:19:44]   GPU model: 1 x NVIDIA GeForce RTX 4090
  0%|          | 0/39 [00:00<?, ?it/s][codecarbon INFO @ 13:

KeyboardInterrupt: 

[codecarbon INFO @ 13:20:32] Energy consumed for RAM : 0.000073 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:20:32] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:20:32] Energy consumed for all CPUs : 0.000594 kWh. All CPUs Power : 47.5 W
[codecarbon INFO @ 13:20:32] 0.000667 kWh of electricity used since the begining.
[codecarbon INFO @ 13:20:47] Energy consumed for RAM : 0.000097 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:20:47] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:20:47] Energy consumed for all CPUs : 0.000792 kWh. All CPUs Power : 47.5 W
[codecarbon INFO @ 13:20:47] 0.000889 kWh of electricity used since the begining.
[codecarbon INFO @ 13:21:02] Energy consumed for RAM : 0.000122 kWh. RAM Power : 5.841164588928223 W
[codecarbon INFO @ 13:21:02] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 13:21:02] Energy consumed f